sfsdfsdおれ

In [1]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 1.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 1.2 MB/s eta 0:00:0000:0100:01


In [ ]:
import pandas as pd

# データAとデータBのCSVファイルのパス
data_a_path = '../data/original_data/race_table/combined.csv'  # データAのCSVファイルパス
data_b_path = '../data/merged_data/merged_sorted.csv'  # データBのCSVファイルパス

# CSVファイルからデータを読み込む
data_a = pd.read_csv(data_a_path,low_memory=False)
data_b = pd.read_csv(data_b_path)

# Code列でデータを結合 (data_bにdata_aの情報を追加)
merged_data = pd.merge(data_b, data_a, on='Code', how='left')
# 不要なデータを除去
merged_data=merged_data.dropna()
merged_data = merged_data.drop(columns=['first_horses'])
merged_data = merged_data.drop(columns=['second_horses'])
merged_data = merged_data.drop(columns=['third_horses'])
merged_data = merged_data.drop(columns=['Time_y'])
merged_data = merged_data.drop(columns=['Jockey_y'])
merged_data = merged_data.drop(columns=['Trainer_y'])
結果の確認
print(merged_data)
# merged_data.to_csv('merged_data_final.csv', index=False)

/tmp/ipykernel_23730/2547677098.py:9: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data_b = pd.read_csv(data_b_path)


: 